# AWS_reviews_count_vectorizer-sentiment_analysis

# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

# Loading the Data

In [ ]:
import pandas as pd
# Setting chunk size for reading the CSV in chunks (useful for large datasets)
chunk_size = 10000
chunks = pd.read_csv("/content/aws_review_sofware_dataset.csv", sep=',', chunksize=chunk_size)

# Get the first chunk and access its columns
df = next(chunks)
print(df.columns)


Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')


In [ ]:
df.columns  # Print the column names of the DataFrame to check the structure of the first chunk

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

# Data Preprocessing

In [ ]:
# Assigning a default value of "default value" to new columns 'words' and 'sentences' in the DataFrame
df["words"] = "default value"
df["sentences"] = "default value"

# Iterating through each row in the DataFrame
for i in range(df.shape[0]):
    # Using df.at to update the 'words' and 'sentences' columns with empty lists
    df.at[i, "words"] = list("")  # Creates an empty list for 'words'
    df.at[i, "sentences"] = list("")  # Creates an empty list for 'sentences'


#  NLTK Tokenization Import


In [ ]:
from nltk.tokenize import sent_tokenize # Importing the sent_tokenize function from the Natural Language Toolkit (nltk)

#  NLTK Download

In [ ]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Sentence Tokenization

In [ ]:
# Loop through each row in the DataFrame
for i in range(df.shape[0]):
    # Tokenize the text in the "reviewText" column of the current row into sentences
    l1 = sent_tokenize(str(df.loc[i, "reviewText"]))

    # Assign the tokenized sentences to the "sentences" column of the current row
    df.at[i, "sentences"] = l1


In [ ]:
!pip install pywsd  # Install the pywsd library for Word Sense Disambiguation

In [ ]:
# Import the Natural Language Toolkit (nltk)
import nltk

# Download the 'averaged_perceptron_tagger' resource for part-of-speech tagging
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# Import the `lemmatize_sentence` function from the pywsd.utils module
from pywsd.utils import lemmatize_sentence


In [ ]:
# Import the Natural Language Toolkit (nltk)
import nltk

# Download the 'wordnet' resource for lexical database access
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Lemmatization

In [ ]:
# Iterate through each row in the DataFrame
for k in range(df.shape[0]):
    # Initialize the 'words' column as an empty list for the current row
    df.at[k, "words"] = list("")

    # Iterate through each sentence in the 'sentences' column of the current row
    for j in range(len(df.loc[k, "sentences"])):
        # Extend the 'words' list with lemmatized words from the current sentence
        df.at[k, "words"].extend(lemmatize_sentence(df.loc[k, "sentences"][j]))


In [ ]:
df["words_sentences"] = "default"  # Initialize the 'words_sentences' column with the value 'default' for all rows

In [ ]:
# Import the functools library
import functools

# Iterate through each row in the DataFrame
for k in range(df.shape[0]):
    # Concatenate all words in the 'words' column into a single string with spaces in between
    df.loc[k, "words_sentences"] = functools.reduce(lambda a, b: str(a) + " " + str(b), df.loc[k, "words"])


# Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer  # Import the CountVectorizer class from sklearn.feature_extraction.text

In [ ]:
# Create a copy of the original DataFrame
df1 = df

# Define the maximum number of features for the CountVectorizer
no_features = 1000

# Initialize the CountVectorizer with specified parameters
tf_vectorizer = CountVectorizer(max_features=no_features, stop_words='english')

# Transform the text data into a matrix of token counts
tf = tf_vectorizer.fit_transform(df1.words_sentences)


In [ ]:
df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())  # Convert the sparse matrix to a DataFrame

In [ ]:
df_y=df["verified"]  # Extract the 'verified' column from the DataFrame

# Label Encoding

In [ ]:
# Import the LabelEncoder class from sklearn.preprocessing
from sklearn.preprocessing import LabelEncoder

In [ ]:

le=LabelEncoder()

In [ ]:
# Convert the 'verified' column to a DataFrame
df_y_1 = pd.DataFrame(df_y)


In [ ]:
df_y_enc=df_y_1.apply(le.fit_transform)  # Encode the categorical labels in df_y_1 using LabelEncoder

In [ ]:
# Display the columns of the encoded DataFrame
df_y_enc.columns


Index(['verified'], dtype='object')

In [ ]:
# Display the first 5 rows of the encoded DataFrame
df_y_enc.head(5)


,verified
0,1
1,1
2,1
3,1
4,0


# Random Forest Classifier

In [ ]:
# Import the RandomForestClassifier and accuracy_score functions
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the RandomForestClassifier with 500 estimators and a random state for reproducibility
rf = RandomForestClassifier(n_estimators=500, random_state=42)

# Train the model on the feature matrix df_x and the encoded target labels df_y_enc
rf.fit(df_x, df_y_enc)

# Calculate the accuracy of the model on the training data
accuracy_rf = rf.score(df_x, df_y_enc)

# Print the accuracy as a percentage
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 99.93%


# Naive Bayes

In [ ]:
# Import the MultinomialNB class from sklearn.naive_bayes
from sklearn.naive_bayes import MultinomialNB

# Initialize the Multinomial Naive Bayes classifier
nb = MultinomialNB()

# Train the Naive Bayes model on the feature matrix df_x and the encoded target labels df_y_enc
nb.fit(df_x, df_y_enc)

# Compute the accuracy of the Naive Bayes model on the training data
accuracy_nb = nb.score(df_x, df_y_enc)

# Print the accuracy as a percentage
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes Accuracy: 70.71%


#  Instantiating Gradient Boosting Classifier


In [ ]:
# Import the GradientBoostingClassifier class from sklearn.ensemble
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
# Initialize the Gradient Boosting Classifier with 1000 estimators
GBC = GradientBoostingClassifier(n_estimators=3000)


# Fitting Gradient Boosting Classifier

In [ ]:
# Train the Gradient Boosting Classifier on the feature matrix df_x and the encoded target labels df_y_enc
gb_c = GBC.fit(df_x, df_y_enc)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Model Scoring and Accuracy

In [ ]:
# Compute the accuracy of the Gradient Boosting Classifier on the training data
gbc_score = GBC.score(df_x, df_y_enc)

# Print the accuracy as a percentage
print(f"gbc_score: {gbc_score * 100:.2f}%")


gbc_score: 94.99%
